In [3]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

### Get location of all conll output files

In [21]:
# SPLIT = 'dev'
SPLIT = 'test'

conll_files = []
for idx in range(10):
    conll_files.append(
        f'/share/data/speech/shtoshni/research/litbank_coref/models/longformer_litbank_cross_val_split_{idx}_max_evals_20_max_segment_len_4096/litbank/test.conll')
    
    json_file = f'/share/data/speech/shtoshni/research/litbank_coref/models/longformer_litbank_cross_val_split_{idx}_max_evals_20_max_segment_len_4096/litbank/perf.json'
    print(f"{idx + 1} & {json.load(open(json_file))['litbank_test']['fscore']}")
    

1 & 77.2
2 & 80.3
3 & 78.7
4 & 79.1
5 & 78.7
6 & 78.7
7 & 80.7
8 & 79.1
9 & 80.8
10 & 78.7


### Concat all Cross Val CoNLLs

In [16]:
output_dir = "/tmp"

conll_output_file = path.join(output_dir, f'{SPLIT}.conll')  
print(conll_output_file)
with open(conll_output_file, "w") as output_w:
    for conll_file in conll_files:
        with open(conll_file) as g:
            for line in g:
                output_w.write(line)
                    
        
                    

/tmp/test.conll


#### Setup coref evaluation script path and Gold CoNLL 

In [17]:
import sys
import subprocess
import re

gold_conll = f"/home/shtoshni/Research/litbank_coref/data/litbank/all.{SPLIT}.conll"
scorer_path = "/home/shtoshni/Research/litbank_coref/lrec2020-coref/reference-coreference-scorers/scorer.pl"

def get_coref_score(metric, path_to_scorer, gold=None, preds=None):
    output=subprocess.check_output(["perl", path_to_scorer, metric, preds, gold]).decode("utf-8")
    output=output.split("\n")[-3]
    matcher=re.search("Coreference: Recall: \(.*?\) (.*?)%	Precision: \(.*?\) (.*?)%	F1: (.*?)%", output)
    if matcher is not None:
        recall=float(matcher.group(1))
        precision=float(matcher.group(2))
        f1=float(matcher.group(3))
    return recall, precision, f1

In [18]:
metrics = ['MUC', 'Bcub', 'CEAFE']

fscore_list = []
for metric in metrics:
    recall, precision, fscore = get_coref_score(metric.lower(), scorer_path, gold_conll, conll_output_file)
    print(f" & {recall:.1f} & {precision:.1f} & {fscore:.1f} ", end="")
    fscore_list.append(fscore)

print(f"& {sum(fscore_list)/len(fscore_list): .1f}")


 & 91.1 & 87.7 & 89.4  & 80.1 & 77.8 & 79.0  & 67.4 & 71.7 & 69.5 &  79.3
